# **Air Pollution and Respiratory Disease Analytics — Lagos case study**

# Project Overview
This project analyzes the relationship between air pollution and respiratory diseases in Lagos, Nigeria — Africa's most populous city with **>20 million** people.

## Objectives
1. Derive a **pollution index** (composite feature of all pollutants)  
2. Monitor air pollution trends over time  
3. Analyze relationship between pollution spikes and hospital respiratory cases  
4. Predict respiratory disease surges using pollution data  
5. Identify high-risk cities, periods, and pollutants  
6. Recommend public health and urban environmental policies  

## Hypotheses to Test
- **H1:** Higher **PM2.5** levels correlate with more respiratory hospital cases  
- **H2:** Cities with higher **industrial indices** have worse air quality  
- **H3:** **Harmattan** season shows spikes in **PM10** and respiratory cases  
- **H4:** Weather conditions (low humidity, high temperatures) worsen pollution impact


In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

## **Data Loading and Initial Inspection**

In [2]:
# Load the datasets

df1 = pd.read_excel("lagos_air_pollution_health_data.xlsx")
df2 = pd.read_excel("lagos_air_pollution_health_data_1.xlsx")

In [3]:
# Check out the shape of our datasets

print(f"Dataset 1 shape: {df1.shape}")
print(f"Dataset 2 shape: {df2.shape}")

Dataset 1 shape: (258420, 17)
Dataset 2 shape: (258420, 16)


In [4]:
# Examine the feature names differences

print("\nDataset 1 columns:", df1.columns.tolist())
print("Dataset 2 columns:", df2.columns.tolist())


Dataset 1 columns: ['City', 'date', 'pm2_5', 'pm10', 'no2', 'so2', 'Unnamed: 6', 'o3', 'hospital_id', 'respiratory_cases', 'avg_age_of_patients', 'weather_temperature', 'weather_humidity', 'wind_speed', 'rainfall_mm', 'population_density', 'industrial_activity_index']
Dataset 2 columns: ['C', 'date', 'pm2_5', 'pm10', 'no2', 'so2', 'o3', 'hospital_id', 'respiratory_cases', 'avg_age_of_patients', 'weather_temperature', 'weather_humidity', 'wind_speed', 'rainfall_mm', 'population_density', 'industrial_activity_index']


In [5]:
# Display first five observations of our first dataset

df1.head()

,City,date,pm2_5,pm10,no2,so2,Unnamed: 6,o3,hospital_id,respiratory_cases,avg_age_of_patients,weather_temperature,weather_humidity,wind_speed,rainfall_mm,population_density,industrial_activity_index
0,AJAH,2021-01-01,39.86,98.09,54.93,29.68,29.68,46.00,HOSP_AJA_10,14.0,48.0,27.7,60.0,1.0,9.3,13683.2,0.77
1,AJAH,2021-01-01,68.06,80.35,58.54,33.05,33.05,48.53,HOSP_AJA_6,16.0,44.9,25.9,52.4,3.2,5.0,15838.7,0.27
2,AJAH,2021-01-01,77.60,76.60,58.08,25.11,25.11,17.85,HOSP_AJA_10,9.0,22.7,27.9,51.9,3.2,8.4,17918.3,0.30
3,AJAH,2021-01-01,90.22,98.94,74.46,19.47,19.47,35.27,HOSP_AJA_7,12.0,41.4,28.9,73.6,3.8,0.3,28635.0,0.51
4,AJAH,2021-01-01,85.23,95.18,40.59,14.81,14.81,52.23,HOSP_AJA_3,17.0,37.0,32.4,70.4,2.2,10.0,24057.7,0.77


In [6]:
# Display first five observations of our second dataset

df2.head()

,C,date,pm2_5,pm10,no2,so2,o3,hospital_id,respiratory_cases,avg_age_of_patients,weather_temperature,weather_humidity,wind_speed,rainfall_mm,population_density,industrial_activity_index
0,I K E J A,2021-01-01,65.64,135.39,45.21,22.29,34.84,HOSP_IKE_1,16.0,44.0,25.1,40.0,3.1,2.2,22937.7,0.87
1,Ikeja,2021-01-01,76.49,116.65,50.01,10.20,32.19,HOSP_IKE_4,12.0,33.1,25.3,53.5,3.8,2.2,21787.8,0.32
2,I K E J A,2021-01-01,55.50,101.72,39.10,21.91,49.99,HOSP_IKE_1,20.0,33.5,27.7,72.3,4.7,1.0,14798.0,0.78
3,Ikeja,2021-01-01,56.37,138.53,46.40,28.13,24.89,HOSP_IKE_8,15.0,49.3,32.8,62.9,4.8,8.2,18013.6,0.24
4,I K E J A,2021-01-01,93.77,123.73,59.55,11.82,29.73,HOSP_IKE_3,20.0,45.4,27.9,64.8,3.4,9.1,25980.3,0.30


In [7]:
# Display basic dataset overview information of our first dataset

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258420 entries, 0 to 258419
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   City                       258392 non-null  object        
 1   date                       258397 non-null  datetime64[ns]
 2   pm2_5                      258417 non-null  float64       
 3   pm10                       258411 non-null  float64       
 4   no2                        258419 non-null  float64       
 5   so2                        258412 non-null  float64       
 6   Unnamed: 6                 258412 non-null  float64       
 7   o3                         258413 non-null  float64       
 8   hospital_id                258418 non-null  object        
 9   respiratory_cases          258413 non-null  float64       
 10  avg_age_of_patients        258404 non-null  float64       
 11  weather_temperature        258416 non-null  float64 

In [8]:
# Display basic dataset overview information of our second dataset

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258420 entries, 0 to 258419
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   C                          258392 non-null  object        
 1   date                       258397 non-null  datetime64[ns]
 2   pm2_5                      258417 non-null  float64       
 3   pm10                       258411 non-null  float64       
 4   no2                        258419 non-null  float64       
 5   so2                        258412 non-null  float64       
 6   o3                         258413 non-null  float64       
 7   hospital_id                258418 non-null  object        
 8   respiratory_cases          258413 non-null  float64       
 9   avg_age_of_patients        258404 non-null  float64       
 10  weather_temperature        258416 non-null  float64       
 11  weather_humidity           258414 non-null  float64 

In [9]:
# Checking for duplicates in the first dataset

df1.duplicated().sum()

0

In [10]:
# Checking for duplicates in the second dataset

df2.duplicated().sum()

0

## **Data Cleaning and PreProcessing**